This notebook should be launched from a session of jupyter notebook that was launched from a DB2 command window
To do this, run an administrator DB2 command window as an administrator and type 'jupyter notebook'

Set up the enviornment:

In [ ]:
import sys,os,os.path
os.environ['IBM_DB_HOME']='C:\Program Files\IBM\SQLLIB'
!pip install ipython-sql
!pip install "ibm-db==2.0.8a" 
!pip install ibm_db_sa

Restart the Kernel if this is your first time installing the above. The next steps will fail unless you do this.

Import the modules and load the SQL magic

In [ ]:
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.dates as mdates
from datetime import datetime
import pandas as pd
from IPython.display import display, HTML, Markdown
import nbextensions

Connect to the database. Change the values of user, host, and password to match your environment. For connection to a local host, use 'localhost' for the host name. Also change the port number and database name in the connection string.

In [ ]:
# Define filename for passwords
filename = 'ember_variables.py'
# source the file
%run $filename

In [ ]:
user=NA1_User
host=NA1_Host
inst=NA1_insts[0]

password=NA1_PW
db=NA1_dbs['sample'][0]
port=NA1_ports['sample']

%sql db2+ibm_db://$user:$password@$host:$port/$db

## Overall System Information

In [ ]:
system_info=%sql SELECT OS_NAME \
    , HOST_NAME \
    , OS_FULL_VERSION \
    , OS_KERNEL_VERSION \
    , OS_ARCH_TYPE \
    , CPU_TOTAL \
    , CPU_ONLINE \
    , CPU_CONFIGURED \
    , CPU_SPEED \
    , CPU_HMT_DEGREE \
    , CPU_CORES_PER_SOCKET \
    , MEMORY_TOTAL \
    , MEMORY_FREE \
    , VIRTUAL_MEM_TOTAL \
    , VIRTUAL_MEM_RESERVED \
    , VIRTUAL_MEM_FREE \
    , CPU_LOAD_SHORT \
    , CPU_LOAD_MEDIUM \
    , CPU_LOAD_LONG \
    , CPU_USAGE_TOTAL \
    , CPU_USER \
    , CPU_IDLE \
    , CPU_IOWAIT \
    , CPU_SYSTEM \
    , SWAP_PAGE_SIZE \
    , SWAP_PAGES_IN \
    , SWAP_PAGES_OUT \
FROM TABLE(SYSPROC.ENV_GET_SYSTEM_RESOURCES()) AS T \
with ur
sys_info_df=system_info.DataFrame()
sys_info_df

## Db2 Version and Fix Pack

In [ ]:
level_info=%sql SELECT INST_NAME \
    , IS_INST_PARTITIONABLE \
    , NUM_DBPARTITIONS \
    , INST_PTR_SIZE \
    , RELEASE_NUM \
    , SERVICE_LEVEL \
    , BLD_LEVEL \
    , PTF \
    , FIXPACK_NUM \
    , NUM_MEMBERS \
FROM SYSIBMADM.ENV_INST_INFO \
with ur
level_info

## Db2 Licensed Product(s)

In [ ]:
level_info=%sql SELECT INSTALLED_PROD \
    , INSTALLED_PROD_FULLNAME \
    , LICENSE_INSTALLED \
    , PROD_RELEASE \
    , LICENSE_TYPE \
from SYSIBMADM.ENV_PROD_INFO \
with ur
level_info

## Db2 Configuration
### Db2 Registry

In [ ]:
registry_settings=%sql SELECT REG_VAR_NAME \
    , REG_VAR_VALUE \
    , IS_AGGREGATE \
    , AGGREGATE_NAME \
    , LEVEL \
from SYSIBMADM.REG_VARIABLES \
with ur
registry_settings

### DBM CFG

In [ ]:
dbm_settings=%sql SELECT NAME \
    , VALUE \
    , VALUE_FLAGS \
    , DEFERRED_VALUE \
    , DEFERRED_VALUE_FLAGS \
from SYSIBMADM.DBMCFG \
with ur
df=dbm_settings.DataFrame()
dbm_df=df.set_index('name')
pd.set_option('display.max_rows', 500)
display(dbm_df)

#### Automatic Checks for DBM CFG

In [ ]:
# Automatic Checks for DBM CFG
# (Sample, build your own)
if int(dbm_df.loc['diaglevel']['VALUE']) == 3:
    print("DIAGLEVEL is 3")
else:
    display(Markdown("**WARNING** DIAGLEVEL is "+str(dbm_df.loc['diaglevel']['VALUE'])+", not 3"))

#### Values to Pay Special Attention to in Manual Verification

In [ ]:
# Values to focus on for manual verification
# (Sample, build your own)
display(Markdown("INTRA_PARALLEL value of "+str(dbm_df.loc['intra_parallel']['VALUE'])))

## DB CFG

In [ ]:
#del db_df
for db in NA1_dbs[inst]:
    print('Database Configuration Information for ',db,' on ',inst)
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    db_settings=%sql SELECT NAME \
        , VALUE \
        , VALUE_FLAGS \
        , DEFERRED_VALUE \
        , DEFERRED_VALUE_FLAGS \
    from SYSIBMADM.DBCFG \
    with ur
    db_df=db_settings.DataFrame()
    db_df=db_df.set_index('name')
    pd.set_option('display.max_rows', 500)
    #display(db_df)
    
    #Database Memory Layout
    bp_sizes=%sql SELECT BP_NAME \
        , pagesize \
        , BP_CUR_BUFFSZ * pagesize /1024 /1024 as cur_bp_size_mb \
        , case when AUTOMATIC = 1 then 'AUTOMATIC' else 'STATIC' end as AUTOMATIC \
    from table(mon_get_bufferpool(null,-2)) mgbp \
        join syscat.bufferpools bp on bp.bpname=mgbp.bp_name \
    where BP_NAME not like 'IBMSYSTEM%' \
    with ur 
    bp_df=bp_sizes.DataFrame()
    bp_df=bp_df.set_index('bp_name')

    display(HTML(df.to_html()))
    
    ## Automatic Checks for this Database
    # (Sample, build your own)
    # IF HADR, then BLOCKNONLOGGED
    if (db_df.loc['hadr_local_host']['VALUE'] is not None and db_df.loc['hadr_remote_host']['VALUE'] is not None and db_df.loc['hadr_local_svc']['VALUE'] is not None and db_df.loc['hadr_remote_svc']['VALUE'] is not None) or (db_df.loc['hadr_target_list']['VALUE'] is not None) :
        if db_df.loc['blocknonlogged']['VALUE'].upper() == 'NO':
            display(Markdown("**WARNING** HADR appears to be configured, but BLOCKNONLOGGED is set to NO."))

    ## Manual Checks for this Database
    # (Sample, build your own)
    # STMT_CONC
    display(Markdown("STMT_CONC is set to "+db_df.loc['stmt_conc']['VALUE']))

    display(db_df)

## Security

In [ ]:
#All databases on this instance
for db in NA1_dbs[inst]:
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    secadm = {}
    secadm.clear()
    dbadm = {}
    dbadm.clear()
    dataaccess = {}
    dataaccess.clear()
    host=NA1_Host
    print('Security Information for ',db,' on ',inst)
    display(Markdown("### Security Information for "+db+" on "+inst))
    secadm_ids=%sql select grantee \
        from syscat.dbauth \
        where securityadmauth='Y' \
        with ur
    display(Markdown("#### IDs with SECADM"))
    display(secadm_ids)
    dbadm_ids=%sql select grantee \
        from syscat.dbauth \
        where dbadmauth='Y' \
        with ur
    display(Markdown("#### IDs with DBADM"))
    display(dbadm_ids)
    dataaccess_ids=%sql select grantee \
        from syscat.dbauth \
        where dataaccessauth='Y' \
        with ur
    display(Markdown("#### IDs with DATAACCESS"))
    display(dataaccess_ids)
    public_perms=%sql select privilege \
            , OBJECTTYPE \
            , OBJECTSCHEMA \
            , OBJECTNAME \
        from sysibmadm.privileges \
        where authid='PUBLIC' \
        order by objecttype, objectschema, objectname \
        with ur
    display(Markdown("#### Privileges held by PUBLIC"))
    public_connect=%sql select connectauth \
        from syscat.dbauth \
        where grantee='PUBLIC' \
        with ur
    pub_df=public_connect.DataFrame()
    if public_connect[0][0] != 'N' :
        display(Markdown("**WARNING** PUBLIC has CONNECT authority on the database"))
    display(public_perms)

## Database Objects

In [ ]:
# Database Objects
## Event Monitors
#All databases on this instance
for db in NA1_dbs[inst]:
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    host=NA1_Host
    display(Markdown("### Event Monitors in "+db+" on "+inst))
    list_evms=%sql select evmonname \
        ,case when event_mon_state(evmonname) = 0 then 'INACTIVE' \
            when event_mon_state(evmonname) = 1 then 'ACTIVE' end as status \
        , target_type \
        , target \
        , autostart \
        , versionnumber \
        from syscat.eventmonitors \
        with ur
    display(list_evms)

In [ ]:
# Database Objects
## Explain Tables
#All databases on this instance
for db in NA1_dbs[inst]:
    expl_schema_last = {}
    expl_schema_valid = {}
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    host=NA1_Host
    display(Markdown("### Explain Tables in "+db+" on "+inst))
    expln_schemas=%sql SELECT tabschema \
        ,count(*) count_tables\
        FROM syscat.tables \
        where tabname like 'EXPLAIN%' \
            or tabname like 'ADVISE%' \
            GROUP BY tabschema \
        with ur
    expln_schema_df=expln_schemas.DataFrame()
    expln_schema_df.set_index('tabschema', inplace=True)
    display(expln_schema_df)

In [ ]:
# Database Objects
## Buffer Pools
#All databases on this instance
for db in NA1_dbs[inst]:
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    host=NA1_Host
    display(Markdown("### Buffer Pools in "+db+" on "+inst))
    list_bps=%sql select \
        substr(bphr.bp_name,1,18) as bp_name \
        , bp_cur_buffsz \
        , pagesize \
        , ((pagesize*bp_cur_buffsz)/1024)/1024 as sz_mb \
        , total_logical_reads \
        , total_physical_reads \
        , data_hit_ratio_percent \
        , (select listagg(tbspace,chr(10)) within group (order by create_time) from syscat.tablespaces ts, syscat.bufferpools b where ts.bufferpoolid = b.bufferpoolid and b.bpname=mgbp.bp_name) as tablespaces \
from sysibmadm.bp_hitratio bphr join table(mon_get_bufferpool(NULL,-2)) mgbp \
        on mgbp.bp_name=bphr.bp_name \
    join syscat.bufferpools sbp on sbp.bpname=mgbp.bp_name \
with ur
    display(list_bps)
    display(Markdown("#### Buffer Pool hit ratios by tablespace"))
    ts_bp_hitratios=%sql select tbsp_name \
        , decimal(((float(pool_data_lbp_pages_found) + float(pool_index_lbp_pages_found) + float(pool_xda_lbp_pages_found) + float(pool_col_lbp_pages_found) - float(pool_async_data_lbp_pages_found) - float(pool_async_index_lbp_pages_found) - float(pool_async_xda_lbp_pages_found) - float(pool_async_col_lbp_pages_found)) / (float(pool_data_l_reads) + float(pool_index_l_reads) + float(pool_xda_l_reads) + float(pool_col_l_reads) + float(pool_temp_data_l_reads) + float(pool_temp_xda_l_reads) + float(pool_temp_index_l_reads) + float(pool_temp_col_l_reads))) * 100,5,2) \
        from table(mon_get_tablespace('',-2)) as t \
        order by tbsp_cur_pool_id \
        with ur
    display(ts_bp_hitratios)

In [ ]:
# Database Objects
## Table Spaces
#All databases on this instance
for db in NA1_dbs[inst]:
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    host=NA1_Host
    display(Markdown("### Table Spaces in "+db+" on "+inst))
    list_tbsps=%sql select  tbsp_name, \
        tbsp_type, \
        tbsp_content_type as type, \
        (select count(*) from syscat.tables st where st.tbspace=t.tbsp_name) as tabcount, \
        tbsp_using_auto_storage as auto_sto, \
        tbsp_auto_resize_enabled as auto_resize, \
        tbsp_page_size as page_size, \
        tbsp_used_pages as used_pages, \
        tbsp_total_pages as total_pages, \
        tbsp_total_pages*tbsp_page_size/1024/1024/1024 as ts_gb, \
        case \
                when tbsp_type = 'SMS' then 'EXCLUDE' \
                when tbsp_using_auto_storage = 1 then 'EXCLUDE' \
                when tbsp_auto_resize_enabled = 1 then 'EXCLUDE' \
                else 'INCLUDE' \
        end as Space_check, \
        case \
                when \
                    tbsp_type = 'DMS' \
                    and tbsp_content_type = 'ANY' \
                    and tbsp_page_size = 4096 \
                then '64' \
                when \
                    tbsp_type = 'DMS' \
                    and tbsp_content_type = 'ANY' \
                    and tbsp_page_size = 8192 \
                then '128' \
                when \
                    tbsp_type = 'DMS' \
                    and tbsp_content_type = 'ANY' \
                    and tbsp_page_size = 16384 \
                then '256' \
                when \
                    tbsp_type = 'DMS' \
                    and tbsp_content_type = 'ANY' \
                    and tbsp_page_size = 32768 \
                then '512' \
                when \
                    tbsp_type = 'DMS' \
                    and tbsp_content_type in ('SYSTEMP','USRTEMP','LARGE') \
                    and tbsp_page_size = 4096 \
                then '8192' \
                when \
                    tbsp_type = 'DMS' \
                    and tbsp_content_type in ('SYSTEMP','USRTEMP','LARGE') \
                    and tbsp_page_size = 8192 \
                then '16384' \
                when \
                    tbsp_type = 'DMS' \
                    and tbsp_content_type in ('SYSTEMP','USRTEMP','LARGE') \
                    and tbsp_page_size = 16384 \
                then '32768' \
                when \
                    tbsp_type = 'DMS' \
                    and tbsp_content_type in ('SYSTEMP','USRTEMP','LARGE') \
                    and tbsp_page_size = 32768 \
                then '65536' \
                else 'EXCLUDE' \
        end as maxsize_thresh \
    from table(mon_get_tablespace('',-2)) as t \
    order by tbsp_name \
    with ur
    ts_df=list_tbsps.DataFrame()
    ts_df=ts_df.set_index('tbsp_name')

    ts_df.plot.pie(y='ts_gb')
    plt.axis('equal')
    plt.legend(bbox_to_anchor=(1,1))
    plt.show()
    display(ts_df)

In [ ]:
# Database Objects
## Indexes
#All databases on this instance
for db in NA1_dbs[inst]:
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    host=NA1_Host
    display(Markdown("### Indexes in "+db+" on "+inst))
    display(Markdown("#### Indexes with a cardinality of one"))
    card1_indexes=%sql select  i.lastused, \
        t.tabschema, \
        t.tabname, \
        indname, \
        colnames, \
        fullkeycard, \
        card, \
        volatile \
    from    syscat.indexes i join syscat.tables t \
        on i.tabname=t.tabname and i.tabschema=t.tabschema \
    where   fullkeycard=1 \
        and indextype not in ('BLOK', 'DIM') \
        and t.tabschema not like 'SYS%' \
        and uniquerule='D' \
        and not exists (select 1 \
                from syscat.references r join syscat.keycoluse k \
                        on r.tabschema=k.tabschema and r.tabname=k.tabname \
                where t.tabschema=r.tabschema \
                        and r.tabname = t.tabname \
                        and k.colname in (      select colname \
                                        from syscat.indexcoluse as ic \
                                        where ic.indschema=i.indschema \
                                        and ic.indname=i.indname)) \
    with ur
    display(card1_indexes)
    display(Markdown("#### Indexes not used in the last 30 days"))
    unused_indexes=%sql with indcols as ( select indschema \
                , indname \
                , listagg(case when colorder = 'A' then '+' when colorder = 'D' then '-' else '>' end || colname,chr(10)) within group (order by colseq) as colnames \
            from syscat.indexcoluse group by indschema, indname) select  i.lastused, \
        t.tabschema, \
        t.tabname, \
        i.indname, \
        ic.colnames, \
        bigint(fullkeycard)as fullkeycard, \
        bigint(card) as table_card, \
        mi.index_scans, \
        mi.index_only_scans, \
        volatile \
    from    syscat.indexes i join syscat.tables t \
        on i.tabname=t.tabname and i.tabschema=t.tabschema \
        join table(mon_get_index('','',-2)) as mi on i.iid=mi.iid and i.tabschema=mi.tabschema and i.tabname = mi.tabname \
        join indcols ic on i.indschema=ic.indschema and i.indname=ic.indname \
    where \
        indextype not in ('BLOK', 'DIM') \
        and t.tabschema not like 'SYS%' \
        and uniquerule='D' \
        and i.lastused < current date - 30 days \
        and card > 0 \
        and not exists (select 1 \
                from syscat.references r join syscat.keycoluse k \
                        on r.tabschema=k.tabschema and r.tabname=k.tabname \
                where t.tabschema=r.tabschema \
                        and r.tabname = t.tabname \
                        and k.colname in (      select colname \
                                        from syscat.indexcoluse as ic \
                                        where ic.indschema=i.indschema \
                                        and ic.indname=i.indname)) \
    with ur
    display(unused_indexes)
    display(Markdown("#### Tables Without any Indexes"))
    no_indexes=%sql select \
        substr(t.tabschema,1,15) as tabschema, \
        substr(t.tabname,1,30) as tabname, \
        lastused, \
        date(stats_time) as stats_date, \
        card \
    from    syscat.tables t \
    where \
        t.tabschema not like 'SYS%' \
        and t.tabname not like 'ADVISE%' \
        and t.tabname not like 'EXPLAIN%' \
        and type = 'T' \
        and t.tabname not like 'TI_%' \
        and not exists (select 1 from syscat.indexes i where t.tabschema=i.tabschema and t.tabname=i.tabname) \
    order by tabschema, tabname \
    with ur 
    display(no_indexes)
    
    display(Markdown("#### Most Used Indexes in the Database"))
    mostused_indexes=%sql with indcols as ( select indschema \
                , indname \
                , listagg(case when colorder = 'A' then '+' when colorder = 'D' then '-' else '>' end || colname,chr(10)) within group (order by colseq) as colnames \
            from syscat.indexcoluse group by indschema, indname) select  i.lastused, \
        t.tabschema as tabschema, \
        t.tabname as tabname, \
        i.indname as indname, \
        ic.colnames, \
        bigint(fullkeycard)as fullkeycard, \
        bigint(card) as table_card, \
        mi.index_scans, \
        mi.index_only_scans, \
        mi.page_allocations, \
        volatile \
    from    syscat.indexes i join syscat.tables t \
        on i.tabname=t.tabname and i.tabschema=t.tabschema \
        join table(mon_get_index('','',-2)) as mi on i.iid=mi.iid and i.tabschema=mi.tabschema and i.tabname = mi.tabname \
        join indcols ic on i.indschema=ic.indschema and i.indname=ic.indname \
    where \
        indextype not in ('BLOK', 'DIM') \
        and t.tabschema not like 'SYS%' \
        and uniquerule='D' \
        and not exists (select 1 \
                from syscat.references r join syscat.keycoluse k \
                        on r.tabschema=k.tabschema and r.tabname=k.tabname \
                where t.tabschema=r.tabschema \
                        and r.tabname = t.tabname \
                        and k.colname in (      select colname \
                                        from syscat.indexcoluse as ic \
                                        where ic.indschema=i.indschema \
                                        and ic.indname=i.indname)) \
    order by mi.index_scans desc \
    fetch first 20 rows only \
    with ur
    display(mostused_indexes)

In [ ]:
# Database Objects
## Tables
#All databases on this instance
for db in NA1_dbs[inst]:
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    host=NA1_Host
    display(Markdown("### Tables in "+db+" on "+inst))
    display(Markdown("#### Tables Not Used in 30 days"))
    unused_tables=%sql select  t.lastused, \
        date(stats_time) as stats_time, \
        date(create_time) as create_time, \
        substr(t.tabschema,1,10) as tabschema, \
        substr(t.tabname,1,25) as tabname, \
        bigint(card) as table_card, \
        mt.table_scans, \
        mt.rows_read, \
        mt.rows_inserted + mt.rows_updated + mt.rows_deleted as rows_altered, \
        t.volatile \
    from    syscat.tables t \
        join table(mon_get_table('','',-2)) as mt on t.tabschema=mt.tabschema and t.tabname = mt.tabname \
    where \
        t.tabschema not like 'SYS%' \
        and t.tabname not like '%EXPLAIN%' \
        and t.tabname not like '%ADVISE%' \
        and t.lastused < current date - 30 days \
        and type = 'T' \
    order by t.lastused, t.card desc, t.tabschema, t.tabname \
    with ur
    display(unused_tables)
    display(Markdown("#### Most used tables"))
    mostused_tables=%sql select  t.lastused, \
        substr(t.tabschema,1,10) as tabschema, \
        substr(t.tabname,1,25) as tabname, \
        bigint(card) as table_card, \
        mt.table_scans, \
        mt.rows_read, \
        case when card >0 then mt.rows_read/card else 0 end as avg_reads_per_row, \
        mt.rows_inserted + mt.rows_updated + mt.rows_deleted as rows_altered, \
        t.volatile \
    from    syscat.tables t \
        join table(mon_get_table('','',-2)) as mt on t.tabschema=mt.tabschema and t.tabname = mt.tabname \
    where \
        t.tabschema not like 'SYS%' \
        and t.tabname not like '%EXPLAIN%' \
        and t.tabname not like '%ADVISE%' \
    order by 7 desc, 8 desc, t.tabschema, t.tabname \
    fetch first 20 rows only \
    with ur 
    display(mostused_tables)
    display(Markdown("#### Largest Tables by Number of Rows"))
    highcard_tables=%sql select  t.lastused, \
        substr(t.tabschema,1,10) as tabschema, \
        substr(t.tabname,1,25) as tabname, \
        bigint(card) as table_card, \
        mt.table_scans, \
        mt.rows_read, \
        case when card >0 then mt.rows_read/card else 0 end as avg_times_each_row_in_table_read, \
        mt.rows_inserted + mt.rows_updated + mt.rows_deleted as rows_altered, \
        t.volatile \
    from    syscat.tables t \
        join table(mon_get_table('','',-2)) as mt on t.tabschema=mt.tabschema and t.tabname = mt.tabname \
    where \
        t.tabschema not like 'SYS%' \
        and t.tabname not like '%EXPLAIN%' \
        and t.tabname not like '%ADVISE%' \
    order by table_card desc, t.tabschema, t.tabname \
    fetch first 20 rows only \
    with ur
    display(highcard_tables)
    display(Markdown("#### Tables making use of the extended row size feature"))
    ext_row_tables=%sql select \
        mgt.tabschema, \
        mgt.tabname \
    from \
        table(mon_get_table(NULL,NULL,NULL)) mgt \
    where \
        mgt.lob_object_l_pages > 0 \
        and not exists (select 1 \
                      from syscat.columns c \
                      where c.tabschema = mgt.tabschema \
              and c.tabname = mgt.tabname \
              and c.typename in ('CLOB','BLOB')) \
    with ur
    display(ext_row_tables)
    

In [ ]:
# Database Objects
## Transaction Logs
#All databases on this instance
for db in NA1_dbs[inst]:
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    host=NA1_Host
    display(Markdown("### Transaction logs in "+db+" on "+inst))
    display(Markdown("#### Transaction log file histogram"))
    archlog_histogram=%sql WITH gen_ts (ts) AS ( \
        VALUES current timestamp - 7 days \
        UNION ALL \
        SELECT ts + 1 hour \
        FROM gen_ts \
        WHERE ts <= current timestamp), \
    format_ts (yyyymmddhh) AS ( \
        SELECT bigint(ts)/10000 \
        FROM gen_ts), \
    log_archives (yyyymmddhh, archive_count) AS ( \
        SELECT substr(start_time, 1, 10) as YYYYMMDDhh, count(*) \
        FROM sysibmadm.db_history \
        WHERE operation = 'X' \
        GROUP BY substr(start_time, 1, 10) ) \
    SELECT \
        translate('ABCD-EF-GH IJh', cast(f.yyyymmddhh as char(12)), 'ABCDEFGHIJ') as hour \
        ,coalesce(a.archive_count,0) AS logs_archived \
    FROM \
       format_ts f \
            LEFT OUTER JOIN log_archives a \
            ON f.yyyymmddhh = a.yyyymmddhh \
    ORDER BY hour \
    with ur
    display(archlog_histogram)
    archlog_df=archlog_histogram.DataFrame()
    # log pages read vs. log pages written
    display(Markdown("#### Log Buffer Sizing"))
    logbuf_sizing=%sql select log_reads \
        , log_writes \
        , decimal(float(log_reads)/float(log_writes),10,5) log_read_write_ratio \
    from table (mon_get_transaction_log(-2)) \
    with ur
    display(logbuf_sizing)

## Database Maintenance

In [ ]:
# Database Maintenance
## Backups
#All databases on this instance
for db in NA1_dbs[inst]:
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    host=NA1_Host
    display(Markdown("### Database Maintenance for "+db+" on "+inst))
    display(Markdown("#### List of all Backups in History"))
    backup_list=%sql select date(timestamp(start_time)) as start_date \
        , time(timestamp(start_time)) as start_time \
        , start_time as start_timestamp \
        , timestampdiff ( 4, varchar(timestamp(end_time) - timestamp(start_time)) ) as duration \
        , case operationtype \
            when 'D' then 'Delta Offline' \
            when 'E' then 'Delta Online' \
            when 'F' then 'Offline' \
            when 'I' then 'Incremental Offline' \
            when 'N' then 'Online' \
            when 'O' then 'Incremental Online' \
         else operationtype \
         end || ' ' || case when objecttype = 'D' then 'DB' else objecttype end as Type \
        , devicetype \
        , sqlcode \
    from sysibmadm.db_history \
    where operation='B' \
    order by start_date, start_time \
    with ur    
    backup_df=backup_list.DataFrame()
    display(backup_df)
## Runstats
    display(Markdown("#### Last runstats date for non-system tables"))
    last_stats=%sql select date(stats_time) as stats_date \
        , count(*) as num_tables \
    from syscat.tables \
    where type='T' \
        and tabschema not like 'SYS%' \
    group by date(stats_time) \
    with ur
    display(last_stats)
    display(Markdown("#### Last runstats date for system tables"))
    last_stats=%sql select month(stats_time) as stats_month \
        , year(stats_time) as stats_year \
        , count(*) as num_tables \
    from syscat.tables \
    where type='T' \
        and tabschema like 'SYS%' \
    group by month(stats_time), year(stats_time) \
    order by stats_year desc, stats_month desc \
    with ur
    display(last_stats)
## Reorgs
    display(Markdown("#### Recent Reorgs"))
    reorg_list=%sql select date(timestamp(start_time)) as start_date \
        , time(timestamp(start_time)) as start_time \
        , tabschema \
        , tabname \
        , case operationtype \
            when 'F' then 'Offline' \
            when 'N' then 'Online' \
         else operationtype \
         end as Type \
        , sqlcode \
    from sysibmadm.db_history \
    where operation='G' \
    order by start_date, start_time \
    with ur
    display(reorg_list)

## Data Pruning

In [ ]:
# Data Pruning
# In seperate Notebook

## SQL Analysis

In [ ]:
# SQL Analysis
# In seperate Notebook

## Memory Picture for Whole Server

### Total Memory on server

In [ ]:
server_memory=%sql select decimal(value/1024,10,2) as mem_tot_gb \
    from sysibmadm.env_sys_resources \
    where name='MEMORY_TOTAL'
server_memory

### Portion of Server Memory this Instance Represents

In [ ]:
instance_memory=%sql select decimal((value*4)/1024/1024,20,2) as instance_memory_gb \
  from sysibmadm.dbmcfg \
  where name='instance_memory' \
  with ur
instance_memory

In [ ]:
#All instances on one server
inst_mem = {}
inst_mem.clear

for inst in NA1_insts:
    port=NA1_ports[inst]
    db=NA1_dbs[inst][0]
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    instance_memory=%sql select decimal((value*4)/1024/1024,20,2) as instance_memory_gb \
    from sysibmadm.dbmcfg \
    where name='instance_memory' \
    with ur
    inst_mem[inst]=instance_memory[0][0]

server_memory=%sql select decimal(value/1024,10,2) as mem_tot_gb \
from sysibmadm.env_sys_resources \
where name='MEMORY_TOTAL'
tot_db2_mem=sum(inst_mem.values())
inst_mem['system']=server_memory[0][0]-tot_db2_mem
labels = inst_mem.keys()
sizes = inst_mem.values()
plt.pie(sizes, labels=labels, autopct='%1.1f%%',  shadow=True, startangle=140)
 
plt.axis('equal')
plt.show()

df = pd.DataFrame.from_dict(data=inst_mem, orient='index')
df.columns = ['GB']
#print(df.to_html())
display(HTML(df.to_html()))

In [ ]:
#All instances on one server - level
level = {}
level.clear()
host=NA1_Host
for inst in NA1_insts:
    port=NA1_ports[inst]
    db=NA1_dbs[inst][0]
    %sql db2+ibm_db://$user:$password@$host:$port/$db
    service_level=%sql select service_level \
    from SYSIBMADM.ENV_INST_INFO \
    with ur
    level[inst]=service_level[0][0]
df = pd.DataFrame.from_dict(data=level, orient='index')
df.columns = ['Service Level']
#print(df.to_html())
display(HTML(df.to_html()))
